In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


Num GPUs Available:  0
Please install GPU version of TF


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.dates as mdates

import tensorflow as tf
import seaborn as sns
from matplotlib import pyplot as plt
import os
from keras import mixed_precision

from datetime import datetime



from keras import backend as K


#import models_base


import models_cuda
import ingest, predict

policy = mixed_precision.Policy('mixed_float16')

mixed_precision.set_global_policy(
    policy
)

## FLAGS ##
# RAS model output or USGS
USGS_FLAG = True

### HENRY RAS ###
csv = r"..\data\Henry_WSS_2017_2023.csv"
columns = {'Q': 'Discharge', 'WSS': 'Slope'}

date = datetime.now().strftime("%B_%d_%H_%M")


target = "Discharge"
data_name = "Henry_RAS_2017_2023_" + f"cuda_testing"#"{date}"



train_range = ["1/1/2017 0:00","12/31/2021 23:45"]
test_range = ["1/1/2022 0:00", "12/31/2022 23:45"]

train_scaled, test_scaled, train_dates, test_dates, all_dates, scaler = ingest.ingest(csv, target, renames= columns, USGS_FLAG=USGS_FLAG, train_range= train_range, test_range= test_range)#train_test_ratio= 0.8)
trainX, trainY = ingest.reshape(train_scaled)#, timestep_type= "hr")
testX, testY = ingest.reshape(test_scaled)#, timestep_type= "hr")


model_names = ['Basic_LSTM']#, "GRU", 'Bidirectional_LSTM', 'Stacked_LSTM']



model = models_cuda.prebuilt_models(model_names[0], trainX, trainY, epochs= 1, 
                                    batch_size=32, loss= "nse", load_models=False, 
                                    data= data_name)

validation_loss = models_cuda.evaluate_model(model, testX, testY)
models_cuda.plot_model(model_names[0], validation_loss, data_name)
K.clear_session()
